얼굴인식 모델 다운

In [ ]:
import dlib
import urllib.request
import os

# 다운로드 링크
url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"

# 파일 다운로드
urllib.request.urlretrieve(url, "shape_predictor_68_face_landmarks.dat.bz2")

# 압축 해제
import bz2

with open("models/shape_predictor_68_face_landmarks.dat", "wb") as f_out, bz2.open("models/shape_predictor_68_face_landmarks.dat.bz2", "rb") as f_in:
    f_out.write(f_in.read())

# 파일 닫기
f_out.close()

# 압축파일 삭제
os.remove("models/shape_predictor_68_face_landmarks.dat.bz2")

#   
# 메인 코드

In [1]:
import cv2  # OpenCV 라이브러리, 이미지 및 비디오 처리에 사용됩니다.

from draw import drawImage # 인식, 출력 등 윈도우 출력 관련 함수 파일 분리



# 카메라 셋팅
capture = cv2.VideoCapture(1)  # 1:디바이스 아이디

# 모델이 예측할 수 있는 영상 크기 설정
cap_width, cap_height = 2560, 1440
capture.set(cv2.CAP_PROP_FRAME_WIDTH, cap_width)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, cap_height)



#------------------------------ 캡쳐 시작!! ------------------------------#
while capture.isOpened():
    ret, img_ori = capture.read()

    if not ret:
        break

    img = img_ori.copy()

    # 메인 함수 결과물 출력
    cv2.imshow('DPS', drawImage(img))
    
    cv2.waitKey(1)  # 1밀리초 대기
    # ESC 키를 누르면 종료
    key = cv2.waitKey(1)
    if key== 27:
        break



# 카메라 종료 및 창 닫기
capture.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


I0000 00:00:1706146147.860732       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-01-25 10:29:08.105 python[53187:4392372] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


1/1 [==============================] - 0s 9ms/step


KeyError: 'latitude'

: 

#   
교통 혼잡도를 경로에 색깔로 출력하는 기능 실패 코드

In [ ]:
import folium
from folium import PolyLine
import json
import requests

url = "https://apis.openapi.sk.com/tmap/routes?version=1&callback=function"

payload = {
    "tollgateFareOption": 16,
    "roadType": 32,
    "endX": 126.87651528717959,
    "endY": 37.49445623129658,
    "reqCoordType": "WGS84GEO",
    "startX": 126.94551528717959,
    "startY": 37.54765623129658,
    "gpsTime": "20191125153000",
    "startName": "Now",
    "endName": "Home",
    "resCoordType": "WGS84GEO",
    "trafficInfo": "Y"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "appKey": "HpVe5crDV38GvHu7tUyMA5CSmhca5PQk3vRWkFH0"
}

response = requests.post(url, json=payload, headers=headers)

# print(response.text)

# TMap API로 받은 데이터 예시 (일부만 사용)
data = json.loads(response.text)

# folium 지도 생성
m = folium.Map(location=[37.54770628011636, 126.94550233876579], zoom_start=15)


# 혼잡도에 따른 색상 매핑
congestion_color_mapping = {
    '0': 'lightblue',
    '1': 'lime',
    '2': 'yellow',
    '3': 'orange',
    '4': 'red',
}

# PolyLine 그리기
for feature in data['features']:
    if feature['geometry']['type'] == 'LineString':
        coordinates = feature['geometry']['coordinates']
        flipped_data = [point[::-1] for point in coordinates]
        # 'traffic' 정보가 'properties'에 있음에 주의
        traffic = feature.get('geometry', {}).get('traffic', [])

        # 혼잡도에 따라 색상 지정
        congestion = '0'
        if str(traffic[0]) == "":
            pass  # 'traffic'가 비어있을 경우 '0'을 그대로 사용
        else:
            congestion = str(traffic[0][2])
        

        print(flipped_data)
        color = congestion_color_mapping.get(congestion, 'black')  # traffic이 비어있을 때 검은색으로 표시
        # PolyLine 그리기
        folium.PolyLine(locations=flipped_data, color=color, weight=10, opacity=1).add_to(m)

# 지도를 HTML 파일로 저장
m.save("tmap_route_map.html")